## This notebook builds the SQLite database used for the datasette instance for air quality data

### Load Libraries

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import sqlalchemy
from sqlite_utils import Database #datasette sqlite utilities

### Download data from air quality monitoring sites

In [9]:
url_aqms = 'https://opendata.bristol.gov.uk/' \
'explore/dataset/air-quality-monitoring-sites/download/' \
'?format=csv&timezone=Europe/London&lang=en&use_labels_for_header=false&csv_separator=%3B'

aqms = pd.read_csv(url_aqms, sep = ";")

Wrangle data from air quality monitoring sites

In [10]:
# change dtype for this field
aqms.colocated = aqms.colocated.fillna(0.0).astype('int', errors = 'ignore')

In [11]:
# df for the splitting of geopoint
latlon_df = aqms.geo_point_2d.str.split(",", n = 1, expand = True)

In [12]:
aqms['latitude'] = latlon_df[0].astype('float')
aqms['longitude'] = latlon_df[1].astype('float') #need long names for cluster map plugin

Create paths for the photos of each monitoring site - not all of these will render

In [13]:
aqms['thumbnail_photo'] = 'http://maps.bristol.gov.uk/media/airquality/thumbs/' + aqms.siteid.astype(str) + '.jpg'
aqms['fullsize_photo'] = 'http://maps.bristol.gov.uk/media/airquality/full/' + aqms.siteid.astype(str) + '.jpg'

In [14]:
aqms.drop(columns = ['geo_point_2d'], inplace = True)
aqms.drop(columns = 'photopath', inplace = True)

In [15]:
aqms.rename(columns = {'siteid': 'site_id'}, inplace = True)

### Download and prep the annual NO2 diffusion tube data

In [16]:
# get the diffusion tube data
url_no2 = 'https://opendata.bristol.gov.uk/' \
'explore/dataset/no2-diffusion-tube-data/download/' \
'?format=csv&timezone=Europe/London&lang=en&use_labels_for_header=false&csv_separator=%3B'

usecols = ['siteid', 'count', 'conc_ugm3', 'year']

no2dt = pd.read_csv(url_no2, sep = ";", usecols = usecols)
no2dt.rename(columns = {'siteid': 'site_id', 'conc_ugm3': 'no2'}, inplace = True)

### Download and prep the continuous data and the raw diffusion tube data - use the ods_api.ipynb notebook

In [17]:
url_aqdc_1 = 'data/aqdc_df_smaller.csv'
path_no2_raw = 'data/raw_tubes.csv'

In [18]:
no2_tubes_raw = pd.read_csv(path_no2_raw, sep = ",")

In [19]:
no2_tubes_raw

,id,site_id,dateon,dateoff,mid_date,no2
0,133942,22,2022-09-30,2022-11-03,2022-10-17,42.2270
1,133970,373,2022-09-30,2022-11-03,2022-10-17,32.4965
2,134083,640,2022-09-30,2022-11-03,2022-10-17,28.2151
3,134089,646,2022-09-30,2022-11-03,2022-10-17,30.1125
4,134094,651,2022-09-30,2022-11-03,2022-10-17,30.9396
...,...,...,...,...,...,...
71390,26440,17,1991-04-01,1991-04-15,1991-04-08,38.0000
71391,26441,18,1991-04-01,1991-04-15,1991-04-08,35.0000
71392,26447,4,1991-04-01,1991-04-15,1991-04-08,105.0000
71393,26449,6,1991-04-01,1991-04-15,1991-04-08,111.0000


In [20]:
# def read_aqdc_from_url(url):
#     drop_cols = ['location', 'geo_point_2d', 'datestart', 'dateend', 'current', 'instrumenttype']
#     df = pd.read_csv(url, sep = ";", usecols = lambda x: x not in drop_cols)
#     #aqdc['date_time'] = pd.to_datetime(aqdc.date_time)
#     df.rename(columns = {'siteid': 'site_id'}, inplace = True)
#     return df
    

Wrangle continuous data to add useful date time artefacts and clean

In [21]:

aqdc = pd.read_csv(url_aqdc_1, sep = ",")
aqdc.drop(columns = 'Unnamed: 0', inplace = True)
aqdc.rename(columns = {'siteid': 'site_id'}, inplace = True)
aqdc['date'] = aqdc['date_time'].str.slice(0, 10)
aqdc['hour'] = aqdc['date_time'].str.slice(11, 13).astype(int)
aqdc['year'] = aqdc['date_time'].str.slice(0, 4).astype(int)
aqdc['month'] = aqdc['date_time'].str.slice(5, 7).astype(int)
aqdc['day_of_month'] = aqdc['date_time'].str.slice(8, 10).astype(int)
aqdc['date_time'] = pd.to_datetime(aqdc.date_time)


In [22]:
aqdc.sort_values(by = ['date_time'], inplace = True, ascending = False)

In [23]:
aqdc

,date_time,site_id,nox,no2,no,pm10,pm25,temp,rh,date,hour,year,month,day_of_month
698895,2022-12-18 10:00:00+00:00,463,18.812094,11.061688,5.053468,NaN,NaN,NaN,NaN,2022-12-18,10,2022,12,18
694413,2022-12-18 10:00:00+00:00,672,44.139438,33.723218,6.731862,NaN,NaN,NaN,NaN,2022-12-18,10,2022,12,18
694412,2022-12-18 10:00:00+00:00,203,23.544468,17.789437,3.748967,NaN,NaN,NaN,NaN,2022-12-18,10,2022,12,18
762036,2022-12-18 10:00:00+00:00,270,35.510344,19.109062,10.701684,NaN,NaN,NaN,NaN,2022-12-18,10,2022,12,18
698810,2022-12-18 10:00:00+00:00,215,77.586936,34.167344,28.306900,NaN,NaN,NaN,NaN,2022-12-18,10,2022,12,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401694,2010-01-01 00:00:00+00:00,203,36.500000,19.750000,4.000000,NaN,NaN,NaN,NaN,2010-01-01,0,2010,1,1
766455,2010-01-01 00:00:00+00:00,447,45.250000,37.750000,7.500000,NaN,NaN,NaN,NaN,2010-01-01,0,2010,1,1
766456,2010-01-01 00:00:00+00:00,459,66.440000,40.570000,16.860000,NaN,NaN,NaN,NaN,2010-01-01,0,2010,1,1
380010,2010-01-01 00:00:00+00:00,395,65.750000,42.000000,15.500000,NaN,NaN,NaN,NaN,2010-01-01,0,2010,1,1


Construct the dictionaries that will be used to populate the tables

In [24]:
# data for inserting as dicts
aqdc_payload = aqdc.to_dict(orient = 'records')
no2dt_payload = no2dt.to_dict(orient = 'records')
aqms_payload = aqms.to_dict(orient = 'records')
no2_tubes_raw_payload = no2_tubes_raw.to_dict(orient = 'records')

Build the database and populate

In [25]:
db = Database('bristol_airquality_datasette.db', recreate = True)

In [26]:
db['no2-diffusion-tube-data'].create({
    'site_id':int,
    'year': int,
    'no2':float,
    'count':int
},
    pk = ['site_id', 'year']
)

<Table no2-diffusion-tube-data (site_id, year, no2, count)>

In [27]:
db['air-quality-monitoring-sites'].create(
{
    'location':str,
   'site_id':int,
   'easting':int,
   'northing':int,
   'current':int,
   'pollutants':str,
   'instrumenttype':str,
   'exposure':int,
   'tube_kerb_distance_m':float,
   'rec_kerb_distance_m':float,
   'comments':str,
   'aqma':int,
   'detailed_location':str,
   'sample_height':float,
   'elevation':float,
   'grid_id':float,
   'locationclass':str,
   'thumbnail_photo':str,
   'fullsize_photo':str, 
   'datestart':str,
   'dateend':str,
   'description':float,
   'colocated':int,
   'duplicate_triplicate':str,
   'laqm_locationclass':str,
   'latitude':float,
   'longitude':float
},
    pk = ['site_id']
)

<Table air-quality-monitoring-sites (location, site_id, easting, northing, current, pollutants, instrumenttype, exposure, tube_kerb_distance_m, rec_kerb_distance_m, comments, aqma, detailed_location, sample_height, elevation, grid_id, locationclass, thumbnail_photo, fullsize_photo, datestart, dateend, description, colocated, duplicate_triplicate, laqm_locationclass, latitude, longitude)>

In [28]:
db["air-quality-data-continuous"].create({
   "site_id": int,
   "date_time": str,
   "date": str,
   "year":int,
   "month": int,
   "day_of_month":int,
   "hour":int,
   "nox": float,
   "no2": float,
   "no": float,
   "pm10": float,
   "pm25": float,
   "temp": float,
   "rh": float
    },
    pk = ("date_time", "site_id")
)

<Table air-quality-data-continuous (site_id, date_time, date, year, month, day_of_month, hour, nox, no2, no, pm10, pm25, temp, rh)>

In [29]:
db['no2-tubes-raw'].create({
   "id":int,
   "site_id": int,
   "dateon": str,
   "dateoff": str,
   "mid_date":str,
   "no2":float
},
pk = "id")

<Table no2-tubes-raw (id, site_id, dateon, dateoff, mid_date, no2)>

This section builds the relationships between the primary and foreign keys in the relevant tables

In [30]:
# table.add_foreign_key(column, other_table, other_column)

db['air-quality-data-continuous'].add_foreign_key('site_id', 'air-quality-monitoring-sites', 'site_id')
db['air-quality-monitoring-sites'].add_foreign_key('site_id', 'air-quality-data-continuous', 'site_id')
db['no2-diffusion-tube-data'].add_foreign_key('site_id', 'air-quality-monitoring-sites', 'site_id')
db['no2-tubes-raw'].add_foreign_key('site_id', 'air-quality-monitoring-sites', 'site_id')


<Table no2-tubes-raw (id, site_id, dateon, dateoff, mid_date, no2)>

Now we insert the payloads into the created tables

In [31]:
db['no2-diffusion-tube-data'].insert_all(no2dt_payload)

<Table no2-diffusion-tube-data (site_id, year, no2, count)>

In [32]:
db['air-quality-monitoring-sites'].insert_all(aqms_payload)

<Table air-quality-monitoring-sites (location, site_id, easting, northing, current, pollutants, instrumenttype, exposure, tube_kerb_distance_m, rec_kerb_distance_m, comments, aqma, detailed_location, sample_height, elevation, grid_id, locationclass, thumbnail_photo, fullsize_photo, datestart, dateend, description, colocated, duplicate_triplicate, laqm_locationclass, latitude, longitude)>

In [33]:
db['air-quality-data-continuous'].insert_all(aqdc_payload)

<Table air-quality-data-continuous (site_id, date_time, date, year, month, day_of_month, hour, nox, no2, no, pm10, pm25, temp, rh)>

In [34]:
db['no2-tubes-raw'].insert_all(no2_tubes_raw_payload)

<Table no2-tubes-raw (id, site_id, dateon, dateoff, mid_date, no2)>

In [35]:
print(db.schema)

CREATE TABLE [no2-diffusion-tube-data] (
   [site_id] INTEGER,
   [year] INTEGER,
   [no2] FLOAT,
   [count] INTEGER,
   PRIMARY KEY ([site_id], [year]),
   FOREIGN KEY([site_id]) REFERENCES [air-quality-monitoring-sites]([site_id])
);
CREATE TABLE [air-quality-monitoring-sites] (
   [location] TEXT,
   [site_id] INTEGER PRIMARY KEY,
   [easting] INTEGER,
   [northing] INTEGER,
   [current] INTEGER,
   [pollutants] TEXT,
   [instrumenttype] TEXT,
   [exposure] INTEGER,
   [tube_kerb_distance_m] FLOAT,
   [rec_kerb_distance_m] FLOAT,
   [comments] TEXT,
   [aqma] INTEGER,
   [detailed_location] TEXT,
   [sample_height] FLOAT,
   [elevation] FLOAT,
   [grid_id] FLOAT,
   [locationclass] TEXT,
   [thumbnail_photo] TEXT,
   [fullsize_photo] TEXT,
   [datestart] TEXT,
   [dateend] TEXT,
   [description] FLOAT,
   [colocated] INTEGER,
   [duplicate_triplicate] TEXT,
   [laqm_locationclass] TEXT,
   [latitude] FLOAT,
   [longitude] FLOAT,
   FOREIGN KEY([site_id]) REFERENCES [air-quality-data

### This section sets up spatial index to enable spatial operations on the data. This is needed for the spatialite functionality

In [36]:
conn = sqlite3.connect("bristol_airquality_datasette.db")
# Lead the spatialite extension:
conn.enable_load_extension(True)

In [37]:
conn.load_extension("/usr/lib/x86_64-linux-gnu/mod_spatialite")

In [38]:
# Initialize spatial metadata for this database:
conn.execute("select InitSpatialMetadata(1)")
# Add a geometry column called point_geom to our museums table:
conn.execute(
    "SELECT AddGeometryColumn('air-quality-monitoring-sites', 'point_geom', 4326, 'POINT', 2);"
)

In [39]:
# Now update that geometry column with the lat/lon points
conn.execute(
    """
    UPDATE 'air-quality-monitoring-sites' SET
    point_geom = GeomFromText('POINT('||"longitude"||' '||"latitude"||')',4326);
"""
)

In [40]:
# Now add a spatial index to that column
conn.execute(
    'select CreateSpatialIndex("air-quality-monitoring-sites", "point_geom");'
)

In [41]:
# If you don't commit your changes will not be persisted:
conn.commit()
conn.close()

Now set up some Views for easy access to commonly desired snapshots of the data

In [42]:
if db['no2_diffusion_tube_locations_vw'].exists():
    db['no2_diffusion_tube_locations_vw'].drop()

In [43]:
db.create_view("no2_diffusion_tube_locations_vw", """
SELECT
    'no2-diffusion-tube-data'.site_id,
    location,
    year,
    round(no2, 1) no2,
    easting,
    northing,
    round(latitude, 6) latitude,
    round(longitude, 6) longitude,
    count
FROM 'air-quality-monitoring-sites'
INNER JOIN 'no2-diffusion-tube-data'
ON 'air-quality-monitoring-sites'.site_id = 'no2-diffusion-tube-data'.site_id
ORDER BY 'no2-diffusion-tube-data'.site_id, year
""")

<Database <sqlite3.Connection object at 0x7f485d3157b0>>

In [44]:
if db["annual_mean_continuous_vw"].exists():
    db["annual_mean_continuous_vw"].drop()

In [45]:
db.create_view("annual_mean_continuous_vw", """
SELECT 
    'air-quality-data-continuous'.site_id,
    year,
    ROUND(AVG(no2), 1) mean_no2,
    ROUND(AVG(pm10), 1) mean_pm10,
    ROUND(AVG(pm25), 1) as mean_pm25
FROM 'air-quality-data-continuous'
GROUP BY 'air-quality-data-continuous'.site_id, year
ORDER BY year, 'air-quality-data-continuous'.site_id
""")

<Database <sqlite3.Connection object at 0x7f485d3157b0>>

## COMPLETE

In [45]:
# if db['annual_mean_locations_vw'].exists():
#     db['annual_mean_locations_vw'].drop()

In [46]:
%who


Database	 aqdc	 aqdc_payload	 aqms	 aqms_payload	 conn	 db	 latlon_df	 no2_tubes_raw	 
no2_tubes_raw_payload	 no2dt	 no2dt_payload	 np	 path_no2_raw	 pd	 sqlalchemy	 sqlite3	 url_aqdc_1	 
url_aqms	 url_no2	 usecols	 


In [45]:
#not working
# db.create_view("annual_mean_locations_vw", """
# SELECT 
#     'annual_mean_continuous_vw'.site_id,
#     'air-quality-monitoring-sites'.location,
#     year,
#     mean_no2,
#     mean_pm10,
#     mean_pm25,
#     ROUND(latitude, 6) latitude,
#     ROUND(longitude, 6) longitude
# FROM 'annual_mean_continuous_vw'
# INNER JOIN 'air-quality-monitoring-sites'
# ON 'air-quality-monitoring-sites'.site_id = 'annual_mean_continuous_vw'.site_id
# """)




In [38]:
# print(db.schema)

In [ ]:
!ls -l

In [ ]:
# for row in db['aqdc.db'].rows:
#     print(row)

In [ ]:
# for row in db.query("select * from aqdc where siteid = 203"):
#     print(row)

In [22]:
db.table_names()

['no2dt', 'aqms', 'aqdc']

In [ ]:
# print(db.schema)

In [ ]:
con = sqlite3.connect('aq_data.db')
cur = con.cursor()

In [ ]:
con.close()

In [ ]:
query = """
SELECT *
FROM aqdc
WHERE site_id = 203
"""
df_aqdc = pd.read_sql_query(query, con, parse_dates = {'date_time':"%Y-%m-%dT%H:%M:%S%z"}) # parse the date col in read operation

In [ ]:
df_aqdc['date_time']

In [ ]:
query = """
SELECT *
FROM no2dt
WHERE site_id = 4 AND year = 2021
"""
df_no2dt = pd.read_sql_query(query, con)

In [ ]:
df_no2dt

In [ ]:
query = """
SELECT *
FROM aqms
WHERE instrumenttype = "Continuous (Reference)" AND current
"""
df_aqms = pd.read_sql_query(query, con)

In [ ]:
df_aqms

In [ ]:
# con.close()

In [ ]:
query = "SELECT* FROM aqdc"
df_aqdc = pd.read_sql_query(query,  con)

In [ ]:
df_aqdc

In [ ]:
!ls -l

In [ ]:
url_aqdc_upsert = 'nov2-5.csv'

In [ ]:
aqdc_upsert = read_aqdc_from_url(url_aqdc_upsert)

In [ ]:
aqdc_upsert

In [ ]:
aqdc_upsert_payload = aqdc_upsert.to_dict(orient = 'records')

In [ ]:
db['aqdc'].upsert_all(aqdc_upsert_payload, pk = ['date_time','site_id'])

In [ ]:
query = """
SELECT *
FROM aqdc
"""
df_aqdc = pd.read_sql_query(query, con, parse_dates = {'date_time':"%Y-%m-%dT%H:%M:%S%z"}) # parse the date col in read operation

In [ ]:
df_aqdc